## tCoIR - Text Analysis
### <span style='color: green'>SETUP </span> Prepare and Setup Notebook <span style='float: right; color: red'>MANDATORY</span>

In [1]:
# Setup
%load_ext autoreload
%autoreload 2

import sys
import pandas as pd

sys.path = list(set(['.', '..']) - set(sys.path)) + sys.path

import common.utility as utility

from beakerx.object import beakerx
from beakerx import *
from IPython.display import display #, set_matplotlib_formats

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import domain_logic_tCoIR as domain_logic

logger = utility.getLogger('corpus_text_analysis')

utility.setup_default_pd_display(pd)


## <span style='color: green'>PREPARE </span> HAL Co-Windows Ratio (CWR)<span style='float: right; color: red'>MANDATORY</span>

\begin{aligned}
nw(x) &= \text{number of sliding windows that contains term $x$} \\
nw(x, y) &= \text{number of sliding windows that contains $x$ and $y$} \\
\\
f(x, y) &= \text{normalized version of nw(x, y)} \\
CWR(x, y) &= \frac{nw(x, y)}{nw(x) + nw(y) - nw(x, y)}\\
\end{aligned}

Term co-occurrence frequencies is calculated in accordance with Hyperspace Analogue to Language (Lund; Burgess, 1996) vector-space model. The computation is specified in detail in section 3.1 in (Chen; Lu, 2011).

- Chen Z.; Lu Y., "A Word Co-occurrence Matrix Based Method for Relevance Feedback"
- Lund, K.; Burgess, C. & Atchley, R. A. (1995). "Semantic and associative priming in high-dimensional semantic space".[Link](https://books.google.de/books?id=CSU_Mj07G7UC).
- Lund, K.; Burgess, C. (1996). "Producing high-dimensional semantic spaces from lexical co-occurrence". doi:10.3758/bf03204766 [Link](https://dx.doi.org/10.3758%2Fbf03204766).



In [2]:
import itertools
import glove
import pandas as pd
import collections

def build_vocab(corpus):
    ''' Iterates corpus and add distict terms to vocabulary '''
    logger.info('Builiding vocabulary...')
    token2id = collections.defaultdict()
    token2id.default_factory = token2id.__len__
    for doc in corpus:
        for term in doc:
            token2id[term]
    logger.info('Vocabulary of size {} built.'.format(len(token2id)))
    return token2id

# See http://www.foldl.me/2014/glove-python/
class GloveVectorizer():
    
    def __init__(self, corpus=None, token2id=None):
        
        self.token2id = token2id
        self._id2token = None
        self.corpus = corpus        
        
    @property
    def corpus(self):
        return self._corpus
    
    @corpus.setter
    def corpus(self, value):
    
        self._corpus = value
        self.term_count = sum(map(len, value or []))
        
        if self.token2id is None and value is not None:
            self.token2id = build_vocab(value)
            self._id2token = None
    
    @property
    def id2token(self):
        if self._id2token is None:
            if self.token2id is not None:
                self._id2token = { v:k for k,v in self.token2id.items() }
        return self._id2token
    
    #def fit(self, sentences, window=2, dictionary=None):
    def fit(self, corpus=None, size=2):  #, distance_metric=0, zero_out_diag=False):
        
        if corpus is not None:
            self.corpus = corpus
            
        assert self.token2id is not None, "Fit with no vocabulary!"
        assert self.corpus is not None, "Fit with no corpus!"
        
        glove_corpus = glove.Corpus(dictionary=self.token2id)
        glove_corpus.fit(corpus, window=size)

        self.nw_xy = glove_corpus.matrix
        
        return self
    
    def cooccurence(self, normalize='size', zero_diagonal=True):
        '''Return computed co-occurrence values'''
        
        matrix = self.nw_xy
        
        #if zero_diagonal:
        #    matrix.fill_diagonal(0)
                
        coo_matrix = matrix #.tocoo(copy=False)
        
        df = pd.DataFrame({
            'x_id': coo_matrix.row,
            'y_id': coo_matrix.col,
            'nw_xy': coo_matrix.data,
            'nw_x': 0,
            'nw_y': 0,
        }).reset_index(drop=True)
        
        df = df.assign(
            x_term=df.x_id.apply(lambda x: self.id2token[x]),
            y_term=df.y_id.apply(lambda x: self.id2token[x])
        )
        
        df = df[['x_id', 'y_id', 'x_term', 'y_term', 'nw_xy', 'nw_x', 'nw_y']]
        
        norm = 1.0
        if normalize == 'size':
            norm = self.term_count
        elif normalize == 'max':
            norm = np.max(coo_matrix)
        elif normalize is None:
            logger.warning('No normalize method specified. Using absolute counts...')
            pass # return as as is..."
        else:
            assert False, 'Unknown normalize specifier'

        df_nw_xy = df.assign(cwr=df.nw_xy / norm)
        
        return df_nw_xy[df_nw_xy.cwr > 0]


In [3]:
import sys, array, collections
import scipy.sparse as sp
import numpy as np
import pandas as pd
import itertools
from glove import Corpus 
sys.path = list(set(['.', '..']) - set(sys.path)) + sys.path

import common.utility as utility

logger = utility.getLogger('corpus_text_analysis')

class HyperspaceAnalogueToLanguageVectorizer():
    
    def __init__(self, corpus=None, token2id=None, tick=utility.noop):
        """
        Build vocabulary and create nw_xy term-term matrix and nw_x term global occurence vector
        
        Parameter:
            corpus Iterable[Iterable[str]]

        """
        self.token2id = token2id
        self._id2token = None
        self.corpus = corpus
        
        self.nw_xy = None
        self.nw_x = None
        self.tick = tick

    @property
    def corpus(self):
        return self._corpus
    
    @corpus.setter
    def corpus(self, value):
    
        self._corpus = value
        self.term_count = sum(map(len, value or []))
        
        if self.token2id is None and value is not None:
            self.token2id = build_vocab(value)
            self._id2token = None
    
    @property
    def id2token(self):
        if self._id2token is None:
            if self.token2id is not None:
                self._id2token = { v:k for k,v in self.token2id.items() }
        return self._id2token
    
    def sliding_window(self, seq, n):
        it = itertools.chain(iter(seq), [None] * n)
        memory = tuple(itertools.islice(it, n+1))
        if len(memory) == n+1:
            yield memory
        for x in it:
            memory = memory[1:] + (x,)
            yield memory

    def fit(self, corpus=None, size=2, distance_metric=0, zero_out_diag=False):
        
        '''Trains HAL for a document. Note that sentence borders (for now) are ignored'''
        
        if corpus is not None:
            self.corpus = corpus
            
        assert self.token2id is not None, "Fit with no vocabulary!"
        assert self.corpus is not None, "Fit with no corpus!"

        nw_xy = sp.lil_matrix ((len(self.token2id), len(self.token2id)), dtype=np.int32)
        nw_x = np.zeros(len(self.token2id), dtype=np.int32)
        
        for terms in corpus:
            
            id_terms = ( self.token2id[size] for size in terms)
            
            self.tick()
            
            for win in self.sliding_window(id_terms, size):
                
                #logger.info([ self.id2token[x] if x is not None else None for x in win])
                
                if win[0] is None:
                    continue
                    
                for x in win:
                    if x is not None:
                        nw_x[x] += 1

                for i in range(1, size+1):

                    if win[i] is None:
                        continue
                        
                    if zero_out_diag:
                        if win[0] == win[i]:
                            continue
                        
                    d = float(i) # abs(n - i)
                    if distance_metric == 0: #  linear i.e. adjacent equals window size, then decreasing by one
                        w = (size - d + 1) # / size
                    elif distance_metric == 1: # f(d) = 1 / d
                        w = 1.0 / d
                    elif distance_metric == 2: # Constant value of 1
                        w = 1

                    #print('*', i, self.id2token[win[0]], self.id2token[win[i]], w, [ self.id2token[x] if x is not None else None for x in win])
                    nw_xy[win[0], win[i]] += w
                    
        self.nw_x = nw_x
        self.nw_xy = nw_xy
        #self.f_xy = nw_xy / np.max(nw_xy)

        return self
    
    def to_df(self):
        columns = [ self.id2token[i] for i in range(0,len(self.token2id))]
        return pd.DataFrame(
            data=self.nw_xy.todense(),
            index=list(columns),
            columns=list(columns),
            dtype=np.float64
        ).T
    
    def cwr(self, direction_sensitive=False, normalize='size'):

        n = self.nw_x.shape[0]
        
        nw = self.nw_x.reshape(n,1)
        nw_xy = self.nw_xy
        
        norm = 1.0
        if normalize == 'size':
            norm = float(self.term_count)
        elif norm == 'max':
            norm = float(np.max(nw_xy))
        elif norm == 'sum':
            norm = float(np.sum(nw_xy))
        
        #nw.resize(nw.shape[0], 1)
        
        self.cwr = sp.lil_matrix(nw_xy / (-nw_xy + nw + nw.T)) #nw.reshape(n,1).T))
        
        if norm != 1.0:
            self.cwr = self.cwr / norm
            
        coo_matrix = self.cwr.tocoo(copy=False)
        df = pd.DataFrame({
            'x_id': coo_matrix.row,
            'y_id': coo_matrix.col,
            'cwr': coo_matrix.data
        }).sort_values(['x_id', 'y_id']).reset_index(drop=True)
        
        df = df.assign(
            x_term=df.x_id.apply(lambda x: self.id2token[x]),
            y_term=df.y_id.apply(lambda x: self.id2token[x])
        )
        
        df_nw_x = pd.DataFrame(self.nw_x, columns=['nw'])
        
        df = df.merge(df_nw_x, left_on='x_id', right_index=True, how='inner').rename(columns={'nw': 'nw_x'})
        df = df.merge(df_nw_x, left_on='y_id', right_index=True, how='inner').rename(columns={'nw': 'nw_y'})
        
        df = df[['x_id', 'y_id', 'x_term', 'y_term', 'cwr']]
        
        return df
    
    def cooccurence2(self, direction_sensitive=False, normalize='size', zero_diagonal=True):
        n = self.cwr.shape[0]
        df = pd.DataFrame([(
                i,
                j,
                self.id2token[i],
                self.id2token[j],
                self.nw_xy[i,j],
                self.nw_x[i],
                self.nw_x[j],
                self.cwr[i,j]
            ) for i,j in itertools.product(range(0,n), repeat=2) if self.cwr[i,j] > 0 ], columns=['x_id', 'y_id', 'x_term', 'y_term', 'nw_xy', 'nw_x', 'nw_y', 'cwr'])
        
        return df    
    
    def cooccurence(self, direction_sensitive=False, normalize='size', zero_diagonal=True):
        '''Return computed co-occurrence values'''
        
        matrix = self.nw_xy
        
        if not direction_sensitive:
            matrix += matrix.T
            matrix[np.tril_indices(matrix.shape[0])] = 0
        else:
            if zero_diagonal:
                matrix.fill_diagonal(0)
                
        coo_matrix = matrix.tocoo(copy=False)
        
        df_nw_x = pd.DataFrame(self.nw_x, columns=['nw'])
        
        df = pd.DataFrame({
            'x_id': coo_matrix.row,
            'y_id': coo_matrix.col,
            'nw_xy': coo_matrix.data
        })[['x_id', 'y_id', 'nw_xy']].sort_values(['x_id', 'y_id']).reset_index(drop=True)
        
        df = df.assign(
            x_term=df.x_id.apply(lambda x: self.id2token[x]),
            y_term=df.y_id.apply(lambda x: self.id2token[x])
        )
        
        df = df.merge(df_nw_x, left_on='x_id', right_index=True, how='inner').rename(columns={'nw': 'nw_x'})
        df = df.merge(df_nw_x, left_on='y_id', right_index=True, how='inner').rename(columns={'nw': 'nw_y'})
        
        df = df[['x_id', 'y_id', 'x_term', 'y_term', 'nw_xy', 'nw_x', 'nw_y']]
        
        norm = 1.0
        if normalize == 'size':
            norm = self.term_count
        elif normalize == 'max':
            norm = np.max(coo_matrix)
        elif normalize is None:
            logger.warning('No normalize method specified. Using absolute counts...')
            pass # return as as is..."
        else:
            assert False, 'Unknown normalize specifier'

        #logger.info('Normalizing for document corpus size %s.', norm)

        df_nw_xy = df.assign(cwr=((df.nw_xy / (df.nw_x + df.nw_y - df.nw_xy)) / norm))

        df_nw_xy.loc[df_nw_xy.cwr < 0.0, 'cwr'] = 0
        df_nw_xy.cwr.fillna(0.0, inplace=True)
        
        return df_nw_xy[df_nw_xy.cwr > 0]

def test_burgess_litmus_test():
    terms = 'The Horse Raced Past The Barn Fell .'.lower().split()
    answer = {
     'barn':  {'.': 4,  'barn': 0,  'fell': 5,  'horse': 0,  'past': 0,  'raced': 0,  'the': 0},
     'fell':  {'.': 5,  'barn': 0,  'fell': 0,  'horse': 0,  'past': 0,  'raced': 0,  'the': 0},
     'horse': {'.': 0,  'barn': 2,  'fell': 1,  'horse': 0,  'past': 4,  'raced': 5,  'the': 3},
     'past':  {'.': 2,  'barn': 4,  'fell': 3,  'horse': 0,  'past': 0,  'raced': 0,  'the': 5},
     'raced': {'.': 1,  'barn': 3,  'fell': 2,  'horse': 0,  'past': 5,  'raced': 0,  'the': 4},
     'the':   {'.': 3,  'barn': 6,  'fell': 4,  'horse': 5,  'past': 3,  'raced': 4,  'the': 2}
    }
    df_answer = pd.DataFrame(answer).astype(np.int32)[['the', 'horse', 'raced', 'past', 'barn', 'fell']].sort_index()
    #display(df_answer)
    vectorizer = HyperspaceAnalogueToLanguageVectorizer()
    vectorizer.fit([terms], size=5, distance_metric=0)
    df_imp = vectorizer.to_df().astype(np.int32)[['the', 'horse', 'raced', 'past', 'barn', 'fell']].sort_index()
    assert df_imp.equals(df_answer), "Test failed"
    #df_imp == df_answer

    # Example in Chen, Lu:
    terms = 'The basic concept of the word association'.lower().split()
    vectorizer = HyperspaceAnalogueToLanguageVectorizer().fit([terms], size=5, distance_metric=0)
    df_imp = vectorizer.to_df().astype(np.int32)[['the', 'basic', 'concept', 'of', 'word', 'association']].sort_index()
    df_answer = pd.DataFrame({
        'the': [2, 5, 4, 3, 6, 4],
        'basic': [3, 0, 5, 4, 2, 1],
        'concept': [4, 0, 0, 5, 3, 2], 
        'of': [5, 0, 0, 0, 4, 3],
        'word': [0, 0, 0, 0, 0, 5],
        'association': [0, 0, 0, 0, 0, 0]
        },
        index=['the', 'basic', 'concept', 'of', 'word', 'association'],
        dtype=np.int32
    ).sort_index()[['the', 'basic', 'concept', 'of', 'word', 'association']]
    assert df_imp.equals(df_answer), "Test failed"
    print('Test run OK')
    
    
    
test_burgess_litmus_test()


2019-06-25 11:26:40,542 : INFO : Builiding vocabulary...
2019-06-25 11:26:40,543 : INFO : Vocabulary of size 7 built.
2019-06-25 11:26:40,550 : INFO : Builiding vocabulary...
2019-06-25 11:26:40,552 : INFO : Vocabulary of size 6 built.


Test run OK


## <span style='color: green'>PREPARE </span> Compute Using Prepared Tokenized Corpus <span style='float: right; color: red'>MANDATORY</span>


In [4]:
import ipywidgets as widgets
import os, glob
import text_corpus
import domain_logic_tCoIR as domain_logic
import time

class PreparedCorpusUserInterface():
    
    def __init__(self, data_folder):
        self.data_folder = data_folder
        
    def display(self, compute_handler):
        
        def on_button_clicked(b):
            
            if self.filepath.value is None:
                return
            
            with self.out:
                self.button.disabled = True
                compute_handler(
                    self.filepath.value,
                    window_size=self.window_size.value,
                    distance_metric=self.distance_metric.value,
                    direction_sensitive=False, # self.direction_sensitive.value,
                    method=self.method.value
                )
                self.button.disabled = False

        corpus_files = sorted(glob.glob(os.path.join(self.data_folder, '*.tokenized.zip')))
        distance_metric_options = [
            ('linear', 0),
            ('inverse', 1),
            ('constant', 2)
        ]
        
        self.filepath            = widgets.Dropdown(description='Corpus', options=corpus_files, value=None, layout=widgets.Layout(width='400px'))
        self.window_size         = widgets.IntSlider(description='Window', min=2, max=40, value=5, layout=widgets.Layout(width='250px'))
        self.method              = widgets.Dropdown(description='Method', options=['HAL', 'Glove'], value='HAL', layout=widgets.Layout(width='200px'))
        self.button              = widgets.Button(description='Compute', button_style='Success', layout=widgets.Layout(width='115px',background_color='blue'))
        self.out                 = widgets.Output()
        
        self.distance_metric     = widgets.Dropdown(description='Dist.f.', options=distance_metric_options, value=2, layout=widgets.Layout(width='200px'))
        #self.direction_sensitive = widgets.ToggleButton(description='L/R', value=False, layout=widgets.Layout(width='115px',background_color='blue'))
        #self.zero_diagonal       = widgets.ToggleButton(description='Zero Diag', value=False, layout=widgets.Layout(width='115px',background_color='blue'))
        
        self.button.on_click(on_button_clicked)
        
        return widgets.VBox([
            widgets.HBox([
                widgets.VBox([
                    self.filepath,
                    self.method
                ]),
                widgets.VBox([
                    self.window_size,
                    self.distance_metric
                ]),
                widgets.VBox([
                    #self.direction_sensitive,
                    self.button
                ])
            ]),
            self.out])
    
import re
def source_corpus_filename(tokenized_corpus_name):
    try:
        m = re.match('(.*\.txt)_preprocessed.*', tokenized_corpus_name)
        return m.groups()[0] + '.zip'
    except:
        return None

def do_some_stuff(
    filepath,
    window_size=5,
    distance_metric=0,
    direction_sensitive=False,
    normalize='size',
    zero_diagonal=True,
    method='HAL'
):

    corpus = text_corpus.SimplePreparedTextCorpus(filepath)
    doc_terms = [ [ t for t in terms if len(t) > 2] for terms in corpus.get_texts() ]
    
    common_token2id = build_vocab(doc_terms)
    
    source_name = source_corpus_filename(filepath)
    document_index = domain_logic.compile_unesco_corpus_index(corpus)
    source_index = domain_logic.load_corpus_index(source_name).set_index('local_number')
    #threshold = 0.005
    
    dfs = []
    # FIXM#:Loop over year instead, and filter out docs for year
    for i in range(0, len(doc_terms)):
        
        doc          = doc_terms[i] 
        local_number = document_index.iloc[i].local_number
        year         = source_index.loc[local_number].year
        
        logger.info('Year %s...', year)
        
        if method == "HAL":
            
            vectorizer = HyperspaceAnalogueToLanguageVectorizer(token2id=common_token2id)\
                .fit([doc], size=window_size, distance_metric=distance_metric)
        
            df = vectorizer.cooccurence(direction_sensitive=direction_sensitive, normalize=normalize, zero_diagonal=zero_diagonal)
        else:
            
            vectorizer = GloveVectorizer(token2id=common_token2id)\
                .fit([doc], size=window_size)
            
            df = vectorizer.cooccurence(normalize=normalize, zero_diagonal=zero_diagonal)
            
        df['year'] = year
        #df = df[df.cwr >= threshhold]
        
        dfs.append(df[['year', 'x_term', 'y_term', 'nw_xy', 'nw_x', 'nw_y', 'cwr']])

        #if i == 5: break
            
    df = pd.concat(dfs, ignore_index=True)

    df['cwr'] = df.cwr / np.max(df.cwr, axis=0)
    #display(df.sort_values('cwr', ascending=False).head(100))

    result_filename = 'result_co_occurrence_{}.txt'.format(time.strftime("%Y%m%d_%H%M%S"))
    
    df.to_csv(result_filename, sep='\t')
    print('Result saved to file {}'.format(result_filename))
    
    print('Now you are ready to do some serious stuff!')
    #return doc_terms
    
display(PreparedCorpusUserInterface(domain_logic.DATA_FOLDER).display(do_some_stuff))



In [5]:
domain_logic.DATA_FOLDER
import glob
print(glob.glob(domain_logic.DATA_FOLDER + "/*.*"))

'../../data'

['../../data/benedict-xvi_curated_20190326.txt_preprocessed.zip', '../../data/unesco_conference_material_corpus.txt.zip', '../../data/term_substitutions.txt', '../../data/benedict-xvi_curated_20190326.txt_preprocessed_en__disable(parser,textcat)_.pkl.bz2', '../../data/unesco_dataset_index.xlsx', '../../data/unesco_conference_material_corpus.txt_preprocessed.zip', '../../data/benedict-xvi_curated_20190326.txt_preprocessed_201903011731.tokenized.zip', '../../data/unesco_courier_corpus.txt_preprocessed_nnp_frq10_ng2.tokenized.zip', '../../data/treaty_text_corpora_20181206.txt_preprocessed.zip', '../../data/unesco_courier_corpus.txt.zip', '../../data/unesco_courier_corpus.txt_preprocessed_nnpadj_frq10_ng2.tokenized.zip', '../../data/tagset.csv', '../../data/benedict-xvi_curated_20190326.txt_preprocessed_en__disable(parser,ner,textcat)_.pkl.bz2', '../../data/benedict-xvi_curated_20190326.txt_preprocessed_en__disable(parser,ner,textcat)_.bin.bz2', '../../data/unesco_courier_corpus_stats.txt'

## <span style='color: green'>PLAYGROUND </span> Ignore  <span style='float: right; color: red'>SKIP</span>


In [ ]:
#corpus = [ current_corpus()[0] ]
#terms = [ list(doc) for doc in textacy_utility.extract_corpus_terms(corpus, extract_args) ]
terms = 'The Horse Raced Past The Barn Fell .'.lower().split()
vectorizer = HyperspaceAnalogueToLanguageVectorizer()
vectorizer.fit(terms, size=5)
print(vectorizer)
matrix = vectorizer.p_ij

matrix += matrix.T
matrix[np.tril_indices(matrix.shape[0])] = 0
coo_matrix = matrix.tocoo(copy=False)
list(coo_matrix.todense())

df_p_i = pd.DataFrame(self.p_i, columns=['p_i_count'])

df = pd.DataFrame({
    'x_id': coo_matrix.row,
    'y_id': coo_matrix.col,
    'p_xy': coo_matrix.data
})[['x_id', 'y_id', 'p_xy']]\
.sort_values(['x_id', 'y_id'])\
.reset_index(drop=True)

df = df.assign(
    x_term=df.x_id.apply(lambda x: self.id2token[x]),
    y_term=df.y_id.apply(lambda x: self.id2token[x])
)

df = df.merge(df_p_i, left_on='x_id', right_index=True, how='inner').rename(columns={'p_i_count': 'p_x'})
df = df.merge(df_p_i, left_on='y_id', right_index=True, how='inner').rename(columns={'p_i_count': 'p_y'})

df = df[['x_id', 'y_id', 'x_term', 'y_term', 'p_xy', 'p_x', 'p_y']]

if normalize_by_term_count:
    df['p_xy'] /= self.term_count
    df['p_x']  /= self.term_count
    df['p_y']  /= self.term_count

df = df.assign(score=df.p_xy / (df.p_x + df.p_y - df.p_xy))     


In [ ]:
df

In [ ]:
import spacy
   
def compute_co_occurrence(corpus, gui, documents, document_filters, terms_filter, window_size, group_by_columns, weighting, tick):
    
    # FIXME: Split based on group_by_filters
    docs = list(gui_utility.get_documents_by_field_filters(corpus, documents, document_filters))
    print(len(docs))
    gui.progress.max = len(docs)
    
    #def terms():
    #    for doc in doc:
    #        gui.progress.value += 1
    #        yield textacy_utility.extract_corpus_terms(docs, terms_filter)
            
    #terms = [ list(doc) for doc in textacy_utility.extract_corpus_terms(docs, terms_filter) ]
    
    vectorizer = HyperspaceAnalogueToLanguageVectorizer(tick=tick)
    vectorizer.fit(terms, size=window_size, weighing=weighting)
    logger.warning('Computation is based on (P_ij + P_ij.T) by default (before/after weights are summed up) )')
    df = vectorizer.cooccurence()
    df.sort_values('score', ascending=False)
    return df.head(500)
    
def display_co_occurrences(gui, df):
    display(df)

def word_co_occurrence_gui(documents, corpus, compute_callback, display_callback, filter_options, group_by_options):
    
    lw = lambda w: widgets.Layout(width=w)
    
    include_pos_tags = [ 'ADJ', 'VERB', 'NUM', 'ADV', 'NOUN', 'PROPN' ]
    weighting_options = { 'Linear': 0, 'Reciprocal': 1, 'Constant': 2 }
    normalize_options = { '':  False, 'Lemma': 'lemma', 'Lower': 'lower' }
    pos_options = include_pos_tags
    
    default_include_pos = ['NOUN', 'PROPN']
    frequent_words = [ x[0] for x in textacy_utility.get_most_frequent_words(corpus, 100, include_pos=default_include_pos) ]

    output_type_options = [ ( 'List', 'table' ), ( 'Rank', 'rank' ) ]
    ngrams_options = { '-': None, '1': [1], '1,2': [1,2], '1,2,3': [1,2,3]}
    
    document_filters = gui_utility.generate_field_filters(documents, filter_options)
    
    gui = types.SimpleNamespace(
        progress=widgets.IntProgress(value=0, min=0, max=5, step=1, description='', layout=lw('98%')),
        document_filters=document_filters,
        ngrams=widgets.Dropdown(description='n-grams', options=ngrams_options, value=[1], layout=lw('200px')),
        normalize=widgets.Dropdown(description='Normalize', options=normalize_options, value='lemma', layout=lw('200px')),
        weighting=widgets.Dropdown(description='Weighting', options=weighting_options, value=0, layout=lw('200px')),
        include_pos=widgets.SelectMultiple(description='POS', options=pos_options, value=default_include_pos, rows=7, layout=lw('150px')),
        stop_words=widgets.SelectMultiple(description='STOP', options=frequent_words, value=list([]), rows=7, layout=lw('200px')),
        group_by_columns=widgets.Dropdown(description='Group by', value=group_by_options[0][1], options=group_by_options, layout=lw('200px')),
        #output_type=widgets.Dropdown(description='Output', value='rank', options=output_type_options, layout=lw('200px')),
        window_size=widgets.IntSlider(description='Window', min=5, max=20, value=5, layout=lw('200px')),
        min_freq=widgets.IntSlider(description='Min freq', min=1, max=10, value=1, layout=lw('200px')),
        compute=widgets.Button(description='Compute', button_style='Success', layout=lw('120px')),
        output=widgets.Output(layout={'border': '1px solid black'})
    )
    
    def tick():
        gui.progress.value += 1
        
    boxes = widgets.VBox([
        gui.progress,
        widgets.HBox([
            widgets.VBox([
                gui.normalize,
                gui.ngrams,
                gui.weighting,
                gui.group_by_columns,
                gui.min_freq,
                gui.window_size
                #gui.output_type,
            ]),
            widgets.VBox([ x['widget'] for x in gui.document_filters]),
            gui.include_pos,
            gui.stop_words,
            widgets.VBox([
                gui.compute,
            ], layout=widgets.Layout(align_items='flex-end')),
        ]),
        gui.output
    ])
    
    display(boxes)
    
    def pos_change_handler(*args):
        with gui.output:
            gui.compute.disabled = True
            selected = set(gui.stop_words.value)
            frequent_words = [
                x[0] for x in textacy_utility.get_most_frequent_words(
                    corpus,
                    100,
                    normalize=gui.normalize.value,
                    include_pos=gui.include_pos.value,
                    weighting=gui.weighting.value
                )
            ]
            gui.stop_words.options = frequent_words
            selected = selected & set(gui.stop_words.options)
            gui.stop_words.value = list(selected)
            gui.compute.disabled = False
        
    gui.include_pos.observe(pos_change_handler, 'value')    
    gui.weighting.observe(pos_change_handler, 'value')    
    
    def compute_callback_handler(*_args):
        gui.output.clear_output()
        with gui.output:
            try:
                gui.compute.disabled = True
                terms_filter = dict(
                    args=dict(
                        ngrams=gui.ngrams.value,
                        named_entities=None,
                        normalize=gui.normalize.value,
                        as_strings=True
                    ),
                    kwargs=dict(
                        min_freq=gui.min_freq.value,
                        include_pos=gui.include_pos.value,
                        filter_stops=True,
                        filter_punct=True
                    ),
                    extra_stop_words=set(gui.stop_words.value),
                    substitutions=None
                )
                df = compute_callback(
                    corpus=corpus,
                    gui=gui,
                    documents=documents,
                    document_filters=[ (x['field'], x['widget'].value) for x in gui.document_filters],
                    terms_filter=terms_filter,
                    window_size=gui.window_size.value,
                    group_by_columns=gui.group_by_columns.value,
                    weighting=gui.weighting.value,
                    tick=tick
                )
                display_callback(gui, df)
            finally:
                gui.compute.disabled = False
                gui.progress.value = 0

    gui.compute.on_click(compute_callback_handler)
    return gui
                
try:
    document_index = domain_logic.compile_documents(current_corpus())
    word_co_occurrence_gui(
        document_index,
        current_corpus(),
        compute_callback=compute_co_occurrence,
        display_callback=display_co_occurrences,
        filter_options=domain_logic.DOCUMENT_FILTERS,
        group_by_options=domain_logic.GROUP_BY_OPTIONS
    )
except Exception as ex:
    raise
    logger.error(ex)
    


In [ ]:
corpus
